#### This code Tests following configuration:

Entity Representation: Words in Entity and Context Words

Type Representation: Similar words to type representation learned on training corpus

Ranking Score: Group Average

Word Representation: Trained word vectors on corpus initialized with pretrained glove vector embeddings

In [1]:
import os, logging, pickle
from operator import itemgetter
import numpy as np
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
entity_ranking_op_folder = '../../Data/output'
type_rep_folder = 'TypeRep/type_word/train'
entity_rep_test_folder = 'EntityRep/test'
entity_context_word_file = 'doc_entity_context_word.p'
type_word_file_name = 'tag_word_dict.p'

word_emnbedding_pretrained_trained_on_corpus = '../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt'

entity_context_word_file = os.path.join(entity_ranking_op_folder, entity_rep_test_folder, entity_context_word_file)
type_word_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_word_file_name)

In [4]:
tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others', 'ORG_Accused', 'ORG_Victim', 'ORG_Others', 'PER_Victim', 'PER_Others', 'PER_Accused']
per_tag_list = ['PER_Victim', 'PER_Others', 'PER_Accused']
loc_tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others']
org_tag_list = ['ORG_Accused', 'ORG_Victim', 'ORG_Others']

In [5]:
entity_rep_context_word_file_dict = pickle.load(open(entity_context_word_file, 'rb'))
type_word_dict = pickle.load(open(type_word_file, 'rb'))
for tag in tag_list:
    print(tag, type_word_dict[tag], '\n')

LOC_Event [('Udgir', 0.7491601705551147), ('Hyderabad', 0.7469158172607422), ('Masjid', 0.7268767356872559), ('December', 0.7225533723831177), ('October', 0.7208514213562012), ('West', 0.7201897501945496), ('Ajmer', 0.7032129764556885), ('Assam', 0.6985844373703003), ('Bengal', 0.6918336153030396), ('February', 0.6780576705932617), ('September', 0.6680309772491455), ('blasts', 0.6621419191360474), ('North', 0.6534678936004639), ('the', 0.6438105702400208), ('three', 0.6426020860671997), ('al-Sadr', 0.6417630910873413), ('bomb', 0.640689492225647), ('RSS', 0.6375901103019714), ('Varanasi', 0.6354883313179016), ('Ahmedabad', 0.6352523565292358), ('Caf', 0.6344596147537231), ('district', 0.6304529905319214), ('2007', 0.622878909111023), ('Kashmir', 0.6182977557182312), ('House', 0.6181615591049194), ('Orissa', 0.6146178245544434), ('blast', 0.6135209202766418), ('Bihar', 0.6130719780921936), ('Baghdad', 0.6105132102966309), ('Peshawar', 0.6065664291381836)] 

LOC_Accused [('terrorist', 0.

In [6]:
word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)
word_vector = word_vectors.get_vector('Narendra')
size = len(word_vector)

2018-05-03 06:03:14,647 : INFO : loading projection weights from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt
2018-05-03 06:05:30,943 : INFO : loaded (408497, 300) matrix from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt


In [7]:
def group_average(entity, role_tuple):
    num_of_words_in_entity = len(entity)
    num_of_words_in_role = len(role)
    total_number_of_words = num_of_words_in_entity + num_of_words_in_role
    
    sum_of_entity_type_word_vec = np.zeros(size)
    for word in entity:
        try:
            word_vec = word_vectors.get_vector(word)
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
            
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
    
    for tup in role_tuple:
        try:
            word_vec = word_vectors.get_vector(tup[0])
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
        
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
        
    dot_product = np.dot(sum_of_entity_type_word_vec, sum_of_entity_type_word_vec)
    group_avg = float(dot_product - (total_number_of_words))/(total_number_of_words * (total_number_of_words - 1))
    return group_avg    
    

In [11]:
precision1 = dict()
doc_count_for_tag = dict()

for tag in tag_list:
    doc_count_for_tag[tag] = 0
    precision1[tag] = 0
    

# Precision at K
K = 1
count = 0
for doc_id in entity_rep_context_word_file_dict:
    #print(doc_id)
    num_actual_entities_with_role = dict()
    if count > 0:
        break
    per_entities = list()
    loc_entities = list()
    org_entities = list()
    doc_role_dict = entity_rep_context_word_file_dict[doc_id]
    for role in doc_role_dict.keys():
        entities = doc_role_dict[role]
        num_actual_entities_with_role[role] = len(entities)
        if role in per_tag_list:
            for entity in entities:
                per_entities.append((entity, role))
        if role in org_tag_list:
            for entity in entities:
                org_entities.append((entity, role))
        if role in loc_tag_list:
            for entity in entities:
                loc_entities.append((entity, role))
    
    num_per_entity = len(per_entities)
    num_org_entity = len(org_entities)
    num_loc_entity = len(loc_entities)
    
    for role in per_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in per_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_per_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_per_entity)
                
    for role in loc_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in loc_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_loc_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_loc_entity)
                
    for role in org_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in org_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_org_entity)):
                
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_org_entity)
    #print(num_actual_entities_with_role)
    #count = 1

for tag in tag_list:
    if doc_count_for_tag[tag] > 0:
        print(tag, float(precision1[tag] * 100)/doc_count_for_tag[tag], '%')

LOC_Event 50.264550264550266 %
LOC_Accused 9.375 %
LOC_Victim 33.333333333333336 %
LOC_Others 66.0 %
ORG_Accused 61.46788990825688 %
ORG_Victim 33.333333333333336 %
ORG_Others 78.0 %
PER_Victim 50.0 %
PER_Others 86.36363636363636 %
PER_Accused 47.61904761904762 %
